## Pruebas


### 1° Lectura

In [1]:
import pandas as pd
import re
# Constantes
TEMA = "ABCDPQRS"
PATRON_CLAVES = 'TRQSP '
PATRON_RESPUESTAS = 'TRQSP *'
LARGO_CODIGO = 8
LARGO_LITO = 7

# Cambiar largo codigo y lito

# Lectura claves
try:
    with open("sdf/cla.sdf", "r") as archivo:
        df_claves = pd.DataFrame(columns=['lito_clave', 'tema_clave', 'solucion'])    
        data_claves = archivo.readlines()
        for line in data_claves:
            lito = line[:6]
            tema = line[6]
            solucion = line[7:].replace('\n', '')
            df_claves = pd.concat([df_claves, pd.DataFrame({'lito_clave': [lito], 'tema_clave': [tema], 'solucion': [solucion]})], ignore_index=True)            
        archivo.close()
    print(df_claves)
except Exception as e:
    print('Hubo un error: ', e)

# Lectura identificadores
try:
    with open("sdf/iden.sdf", "r") as archivo:
        df_identifi = pd.DataFrame(columns=['lito', 'tema', 'codigo'])
        data_identifi = archivo.readlines()
        for linea in data_identifi:
            lito = linea[:6]
            tema = linea[6]
            codigo = linea[7:].replace('\n', '')
            df_identifi = pd.concat([df_identifi, pd.DataFrame({'lito': [lito], 'tema': [tema], 'codigo': [codigo]})], ignore_index=True)
        archivo.close()
    print(df_identifi)
except Exception as e:
    print('Hubo un error: ', e)

# Lectura respuestas
try:
    with open("sdf/res.sdf", "r") as archivo:
        df_respuestas = pd.DataFrame(columns=['lito', 'tema', 'respuesta'])
        data_respuestas = archivo.readlines()
        for linea in data_respuestas:
            lito = linea[:6]
            tema = linea[6]
            respuesta = linea[7:].replace('\n', '')
            df_respuestas = pd.concat([df_respuestas, pd.DataFrame({'lito': [lito], 'tema': [tema], 'respuesta': [respuesta]})], ignore_index=True)
        archivo.close()
    print(df_respuestas)
except Exception as e:
    print('Hubo un error: ', e)



  lito_clave tema_clave solucion
0     033872          A    TRRQR
1     033871          B    TTPTR
2     033874          C    PQQSQ
3     033873          D    PT PR
4     033880          P    TPPR 
5     033877          Q    PTRQP
6     033876          R    QPTSP
7     033875          S    QTRTS
     lito tema    codigo
0  010486    C  72856468
1  010515    D  75701424
2  010484    P  77050605
3  010498    D  72198387
4  010512    B  75959712
     lito tema respuesta
0  010486    C      QR*S
1  010515    D     Q QRQ
2  010484    P     *Q QS
3  010498    D     QQRQS
4  010512    B     QQRQS


### 2° Pruebas de validacion

In [2]:
df_vacantes = pd.read_excel("VACANTES_ADMI2.xlsx")

In [3]:
df_vacantes

,ADMISION,CEPU,TVACANTES_ORDINARIAS,PRIMEROS_PUESTOS,DEPORTISTAS,VICTIMAS_TERRORISMO,TITULADO,TRASLADO_INTERNO,TRASLADO_EXTERNO,PERSONA_DISCAPACIDAD,TOTAL_VACANTES_EXTRAORDINARIAS,TOTAL_GENERAL
0,58,38,96,8,2,2,2,2,2,6,24,120
1,47,31,78,7,2,2,2,2,2,5,22,100
2,17,11,28,3,1,1,1,1,1,2,10,38
3,18,12,30,3,1,1,1,1,1,2,10,40
4,54,36,90,8,2,2,2,2,2,6,24,114
5,33,23,56,5,2,2,2,2,2,4,19,75
6,33,21,54,5,1,1,1,1,1,3,13,67
7,33,21,54,5,1,1,1,1,1,3,13,67
8,33,21,54,5,1,1,1,1,1,3,13,67
9,33,21,54,5,1,1,1,1,1,3,13,67


#### - Validacion de estructura

In [12]:
def estructure_solution(df_claves, df_identifi, df_respuestas, tema, patron_claves, patron_respuestas, largo_codigo, largo_lito):
    # cambia el indice
    df_claves = df_claves.reset_index(drop=True)
    df_claves.index += 1

    df_identifi = df_identifi.reset_index(drop=True)
    df_identifi.index += 1

    df_respuestas = df_respuestas.reset_index(drop=True)
    df_respuestas.index += 1

    men = 'Verificando claves'
    men += '\n'

    def validar_solucion_respuesta(solucion,patron):
        if len(solucion) != 100:
            return False
        if re.match(r'^['+patron+' ]*$', str(solucion)) is None:
            return False
        return True

    # Verificando lito
    lito_clave_esnum = pd.to_numeric(df_claves['lito_clave'], errors='coerce').notnull()
    invalid_rows = df_claves[~lito_clave_esnum]
    if invalid_rows.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en lito\n' + invalid_rows['lito_clave'].to_string(header=False) + '\n'

    # Verificando tema
    valid_tema = df_identifi['tema'].isin([i for i in tema])
    invalid_rows = df_identifi[~valid_tema]
    if invalid_rows.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en lito\n' + invalid_rows['tema'].to_string(header=False) + '\n'

    # Verificando solucion

    valid_rows = df_claves['solucion'].apply(validar_solucion_respuesta, args=(patron_claves,))
    invalid_rows = df_claves[~valid_rows]

    if invalid_rows.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en solucion\n' + invalid_rows['solucion'].to_string(header=False) + '\n'
            

    men += 'Verificando identificaciones'
    men += '\n'

    # Verificando identificaciones
    # Verificando lito
    valid_lito_identifi = pd.to_numeric(df_identifi['lito'], errors='coerce').notnull()
    invalid_rows_lito_identifi = df_identifi[~valid_lito_identifi]
    if invalid_rows_lito_identifi.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en lito\n' + invalid_rows_lito_identifi['lito'].to_string(header=False) + '\n'
    
    # Verificando tema
    valid_tema_identifi = df_identifi['tema'].isin([i for i in tema])
    invalid_rows_tema_identifi = df_identifi[~valid_tema_identifi]
    if invalid_rows_tema_identifi.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en tema\n' + invalid_rows_tema_identifi['tema'].to_string(header=False) + '\n'
    
    # Verificando codigo
    valid_codigo_identifi = pd.to_numeric(df_identifi['codigo'], errors='coerce').notnull()
    valid_codigo_identifi_len = df_identifi['codigo'].apply(lambda x: len(x) == largo_codigo)
    invalid_codigo_identifi = df_identifi[~valid_codigo_identifi]
    invalid_codigo_identifi_len  = df_identifi[~valid_codigo_identifi_len]
    invalid_rows_codigo_identifi = df_identifi[~(valid_codigo_identifi & valid_codigo_identifi_len)]
    if invalid_rows_codigo_identifi.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en tema\n' + invalid_rows_codigo_identifi['codigo'].to_string(header=False) + '\n'

    men += 'Verificando Respuestas'
    men += '\n'

    # Verificando lito
    valid_lito_respuestas = pd.to_numeric(df_respuestas['lito'], errors='coerce').notnull()
    valid_codigo_identifi_len = df_identifi['lito'].apply(lambda x: len(x) == largo_lito)
    invalid_lito_respuesta = df_respuestas[~valid_lito_respuestas]
    if invalid_lito_respuesta.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en tema\n' + invalid_lito_respuesta['lito'].to_string(header=False) + '\n'

    # Verificando tema
    valid_tema_respuestas = df_respuestas['tema'].isin([i for i in tema])
    invalid_rows_tema_respuestas = df_respuestas[~valid_tema_respuestas]
    if invalid_rows_codigo_identifi.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en tema\n' + invalid_rows_codigo_identifi['tema'].to_string(header=False) + '\n'

    # Verificando respuesta
    valid_rows_respuestas = df_respuestas['respuesta'].apply(validar_solucion_respuesta, args=(patron_respuestas,))
    invalid_rows_respuestas = df_respuestas[~valid_rows_respuestas]

    if invalid_rows_respuestas.empty:
        men += 'Hecho\n'
    else:
        men += 'Error en respuesta\n' + invalid_rows_respuestas['respuesta'].to_string(header=False) + '\n'
    return men


In [13]:
val1 = estructure_solution(df_claves, df_identifi, df_respuestas, TEMA, PATRON_CLAVES, PATRON_RESPUESTAS, LARGO_CODIGO, LARGO_LITO)
print(val1)

Verificando claves
Hecho
Hecho
Error en solucion
1    TRRQR
2    TTPTR
3    PQQSQ
4    PT PR
5    TPPR 
6    PTRQP
7    QPTSP
8    QTRTS
Verificando identificaciones
Hecho
Hecho
Hecho
Verificando Respuestas
Hecho
Hecho
Error en respuesta
1     QR*S
2    Q QRQ
3    *Q QS
4    QQRQS
5    QQRQS



#### - Validación duplicados en codigo identificador

In [14]:
def duplicated_code_solution(df_identifi):
    res = ""
    duplicados = df_identifi.duplicated(subset=['codigo'], keep=False)
    if duplicados.any():
        res += "Se encontraron codigos duplicados:\n\n"
        datos_duplicados = {}
        for codigo, df_codigo in df_identifi[duplicados].groupby('codigo'):
            if len(df_codigo) > 1:
                datos_duplicados[codigo] = {'lito': [], 'tema': [], 'indices': []}
                for idx, fila in enumerate(df_codigo.itertuples(), start=1):
                    datos_duplicados[codigo]['lito'].append(fila.lito)
                    datos_duplicados[codigo]['tema'].append(fila.tema)
                    datos_duplicados[codigo]['indices'].append(fila.Index)
                df_duplicados = pd.DataFrame({'lito': datos_duplicados[codigo]['lito'], 'tema': datos_duplicados[codigo]['tema'], 'codigo': [codigo]*len(df_codigo), 'fila': datos_duplicados[codigo]['indices']})
                df_duplicados = df_duplicados[['lito', 'tema', 'codigo', 'fila']]
                res += f"{str(df_duplicados)}\n"
    else:
        res += "No se encontraron duplicados en la columna 'codigo'"
    return res

In [15]:
val2 = duplicated_code_solution(df_identifi)
print(val2)

No se encontraron duplicados en la columna 'codigo'


#### - Validación duplicado de lito en identificador y respuesta

In [16]:
# Solo se debe devolver un resultado
def duplicated_litio_identifi(df_identifi):
    # cambia el indice
    df_identifi = df_identifi.reset_index(drop=True)
    df_identifi.index += 1

    # Verifica si hay duplicados en la columna 'lito'
    duplicados = df_identifi.duplicated(subset=['lito'], keep=False)

    # Filtra las filas que contienen duplicados
    filas_con_duplicados = df_identifi.where(duplicados).dropna()
    

    if filas_con_duplicados.empty:
        return "Duplicado litio identificador\nNo hay duplicados"
    else:
        return "Duplicado litio identificador\n{filas_con_duplicados}"
def duplicated_litio_solution(df_respuestas):
    # cambia el indice
    df_respuestas = df_respuestas.reset_index(drop=True)
    df_respuestas.index += 1

    # Verifica si hay duplicados en la columna 'lito'
    duplicados = df_respuestas.duplicated(subset=['lito'], keep=False)

    # Filtra las filas que contienen duplicados
    filas_con_duplicados = df_respuestas.where(duplicados).dropna()
    

    if filas_con_duplicados.empty:
        return "Duplicado litio respuesta\nNo hay duplicados"
    else:
        return "Duplicado litio respuesta\n{filas_con_duplicados}"

In [17]:
val4_1 = duplicated_litio_identifi(df_identifi)
print(val4_1)
val4_2 = duplicated_litio_solution(df_respuestas)
print(val4_2)

Duplicado litio identificador
No hay duplicados
Duplicado litio respuesta
No hay duplicados


### - Validar lito no localizado

In [18]:
def lito_not_located1(df_identifi, df_respuestas):
    # Unir los dataframes y filtrar las filas no localizadas
    merged = pd.merge(df_identifi, df_respuestas, on=['lito', 'tema'], how='outer', indicator=True)
    no_localizados = merged.query("_merge == 'right_only'").loc[:, ['lito']]

    # Mostrar los resultados
    if no_localizados.empty:
        return "Todos han sido localizados de identificador a respuesta"
    else:
        return f"DataFrame no localizado de identificador a respuesta\n{pd.DataFrame({'lito': no_localizados['lito']})}"

def lito_not_located2(df_identifi, df_respuestas):

    # Unir los dataframes y filtrar las filas no localizadas
    merged = pd.merge(df_respuestas, df_identifi, on=['lito', 'tema'], how='outer', indicator=True)
    no_localizados = merged.query("_merge == 'right_only'").loc[:, ['lito']]

    # Mostrar los resultados
    if no_localizados.empty:
        return "Todos han sido localizados de respuestas a identificador"
    else:
        return f"DataFrame no localizado de respuestas a identificador\n{pd.DataFrame({'lito': no_localizados['lito']})}"


In [19]:
val5_1 = lito_not_located1(df_identifi, df_respuestas)
print(val5_1)
val5_2 = lito_not_located2(df_identifi, df_respuestas)
print(val5_2)

Todos han sido localizados de identificador a respuesta
Todos han sido localizados de respuestas a identificador


### - Lectura base de datos en excel

In [37]:
df = pd.read_csv('postulantes.csv')

In [38]:
df

,codigo,tema,nombres,apellidos,escuela
0,350025,C,alumno_1,apellido_1,escuela_1
1,306126,D,alumno_2,apellido_2,escuela_2
2,326710,P,alumno_3,apellido_3,escuela_3
3,302174,D,alumno_4,apellido_4,escuela_4
4,320804,B,alumno_5,apellido_5,escuela_5
5,316323,B,alumno_6,apellido_6,escuela_6
6,318485,S,alumno_7,apellido_7,escuela_7
7,312488,D,alumno_8,apellido_8,escuela_8
8,345033,Q,alumno_9,apellido_9,escuela_9
9,308296,Q,alumno_10,apellido_10,escuela_10


In [22]:
df_identifi

,lito,tema,codigo
0,010486,C,72856468
1,010515,D,75701424
2,010484,P,77050605
3,010498,D,72198387
4,010512,B,75959712


### OBTENER LOS REGISTROS AUSENTES

In [23]:
# Convertir la columna 'CODIGO' a tipo object
df['CODIGO'] = df['CODIGO'].astype(str)

# Hacer el left anti join
merged = pd.merge(df, df_identifi[['codigo']], how='left', left_on='CODIGO', right_on='codigo', indicator=True)

# Filtrar los registros que no están en df_identifi
result = merged[merged['_merge'] == 'left_only'][df.columns]

# Imprimir el resultado
print(result)

     CODIGO AP_PATERNO AP_MATERNO       NOMBRE   CARRERA
4  77050305      ARIAS    MONTOYA  LUIS FELIPE  SISTEMAS
6  97050805      ARIAS    MONTOYA       ALVARO  SISTEMAS


### OBTENER LOS REGISTROS ANULADOS

In [24]:
# # Revisando estructura
# men=""
# # Verificando codigo    
# valid_codigo_identifi = pd.to_numeric(df_identifi['codigo'], errors='coerce').notnull()
# valid_codigo_identifi_len = df_identifi['codigo'].apply(lambda x: len(x) == LARGO_CODIGO)
# invalid_codigo_identifi = df_identifi[~valid_codigo_identifi]
# invalid_codigo_identifi_len  = df_identifi[~valid_codigo_identifi_len]
# invalid_rows_codigo_identifi = df_identifi[~(valid_codigo_identifi & valid_codigo_identifi_len)]
# if invalid_rows_codigo_identifi.empty:
#     men += 'Hecho\n'
# else:
#     men += 'Error en tema\n' + invalid_rows_codigo_identifi['codigo'].to_string(header=False) + '\n'

# print(men)


### VALIDACIONES UNITARIAS

In [25]:
# LARGO DEL REGISTRO GENERAL

## PARA DF_RESPUESTAS
longitud_lito_respuestas = len(df_respuestas['lito'])
longitud_tema_respuestas = len(df_respuestas['tema'])
longitud_respuestas_respuestas = len(df_respuestas['respuesta'])

print(f"Longitud lito de respuestas: {longitud_lito_respuestas}\nLongitud tema de respuestas: {longitud_tema_respuestas}\nLongitud respuestas de respuesta: {longitud_respuestas_respuestas}")

## PARA DF_IDENTIFI
longitud_lito_iden = len(df_identifi['lito'])
longitud_tema_iden = len(df_identifi['tema'])
longitud_codigo_iden = len(df_identifi['codigo'])

print(f"Longitud lito de respuesta: {longitud_lito_iden}\nLongitud tema de respuesta: {longitud_tema_iden}\nLongitud codigo de respuesta: {longitud_codigo_iden}")


## PARA DF_CLAVE
longitud_lito_iden = len(df_claves['lito_clave'])
longitud_tema_iden = len(df_claves['tema_clave'])
longitud_codigo_iden = len(df_claves['solucion'])

print(f"Longitud lito de respuesta: {longitud_lito_iden}\nLongitud tema de respuesta: {longitud_tema_iden}\nLongitud codigo de respuesta: {longitud_codigo_iden}")


# VALIDACION POR FILA O REGISTRO

## Para df_identifi
numero_registro1= 1 # El indice empieza en 0

fila = df_identifi.iloc[numero_registro1]
longitud_lito = len(fila['lito'])
longitud_tema = len(fila['tema'])
longitud_codigo = len(str(fila['codigo']))

# El indice se muestra apartir de 1
print(f'Para la fila {numero_registro1 + 1} del df_identificador: longitud_lito = {longitud_lito}, longitud_tema = {longitud_tema}, longitud_codigo = {longitud_codigo}')

## Para df_Respuestas
numero_registro2= 1 # El indice empieza en 0

fila = df_respuestas.iloc[numero_registro2]
longitud_lito = len(fila['lito'])
longitud_tema = len(fila['tema'])
longitud_respuesta = len(str(fila['respuesta']))

# El indice se muestra apartir de 1
print(f'Para la fila {numero_registro2 + 1} del df_respuesta: longitud_lito = {longitud_lito}, longitud_tema = {longitud_tema}, longitud_respuesta = {longitud_respuesta}')


## Para df_claves
numero_registro3= 1 # El indice empieza en 0

fila = df_claves.iloc[numero_registro3]
longitud_lito_clave = len(fila['lito_clave'])
longitud_tema_clave = len(fila['tema_clave'])
longitud_solucion = len(str(fila['solucion']))

# El indice se muestra apartir de 1
print(f'Para la fila {numero_registro3 + 1} del df_clave: longitud_lito_clave = {longitud_lito_clave}, longitud_tema_clave = {longitud_tema_clave}, longitud_solucion = {longitud_solucion}')



Longitud lito de respuestas: 5
Longitud tema de respuestas: 5
Longitud respuestas de respuesta: 5
Longitud lito de respuesta: 5
Longitud tema de respuesta: 5
Longitud codigo de respuesta: 5
Longitud lito de respuesta: 8
Longitud tema de respuesta: 8
Longitud codigo de respuesta: 8
Para la fila 2 del df_identificador: longitud_lito = 6, longitud_tema = 1, longitud_codigo = 8
Para la fila 2 del df_respuesta: longitud_lito = 6, longitud_tema = 1, longitud_respuesta = 5
Para la fila 2 del df_clave: longitud_lito_clave = 6, longitud_tema_clave = 1, longitud_solucion = 5


In [26]:
# VALIDACION DE CALIFICACION


## CALIFICAR

In [4]:
def qualify_normal(df_claves, df_identifi, df_respuestas):
    # Constantes para calificar
    correcta = 20
    incorrecta = 1.250
    nula = 0
    puntaje = 0

    # DataFrame con los resultados
    df_result = pd.DataFrame(columns=['lito', 'tema', 'puntaje', 'correctas', 'incorrectas', 'vacias'], index=range(len(df_respuestas)))

    for i in range(len(df_respuestas)):
        var_respuesta = df_respuestas.iloc[i, 2]
        puntaje = 0
        
        # Contadores de correctas, incorrectas, nulas
        num_cor = 0
        num_inc = 0
        num_bla = 0

        # Un bucle para comparar las respuestas con las claves
        for j in range(len(df_claves)):
            var_claves = df_claves.iloc[j, 2]
            if df_respuestas.iloc[i,1] == df_claves.iloc[j,1]:
                for k in range(len(var_respuesta)):
                    if var_claves[k] == " ":
                        puntaje += correcta
                    elif var_respuesta[k] == var_claves[k]: 
                        puntaje += correcta
                        num_cor += 1
                    elif var_respuesta[k] == " ":
                        puntaje += nula
                        num_bla += 1
                    else:
                        puntaje -= incorrecta
                        num_inc += 1
        df_result.iloc[i] = [df_respuestas.iloc[i,0], df_respuestas.iloc[i,1], puntaje, num_cor, num_inc, num_bla]

    # Se hace un merge de DataFrames -> identifi & result
    df_final = pd.merge(df_identifi, df_result, on=['lito', 'tema'])
    
    # Se cambia el orden a descendente -> puntaje
    df_final = df_final.sort_values('puntaje', ascending=False)

    print(df_final)
    # Se crean grupos de DataFrames
    groups = df_final.groupby(df_final.tema)
    # df_final.to_csv('General.csv', index=False, sep=",")
    return df_final

In [5]:
resultado_general = qualify_normal(df_claves, df_identifi, df_respuestas)

     lito tema    codigo puntaje correctas incorrectas vacias
0  010486    C  72856468   16.25         1           3      1
1  010515    D  75701424   16.25         0           3      1
2  010484    P  77050605   16.25         0           3      1
3  010498    D  72198387    15.0         0           4      0
4  010512    B  75959712   -6.25         0           5      0


In [6]:
resultado_general

,lito,tema,codigo,puntaje,correctas,incorrectas,vacias
0,010486,C,72856468,16.25,1,3,1
1,010515,D,75701424,16.25,0,3,1
2,010484,P,77050605,16.25,0,3,1
3,010498,D,72198387,15.0,0,4,0
4,010512,B,75959712,-6.25,0,5,0


### AGREGAR POSTULANTE